# Lab 12. Spark Structured Streaming.

Structured Streaming to skalowalny i odporny na błędy silnik przetwarzania strumieniowego oparty na silniku Spark SQL. Jest to kolejna generacja rozwiązania przetwarzania strumieniowego z wykorzystaniem środowiska Spark, które zostało przedstawione wraz z wersją 2.0 środowiska Spark i jest następcą Spark Streaming, które to nie otrzymuje już żadnych nowych update-ów (możesz poczytać o tej starszej wersji [**tu**](https://spark.apache.org/docs/3.5.1/streaming-programming-guide.html)).

Structured Streaming pozwala na pracę z interfejsami API Dataset oraz DataFrame, które zostały już przedstawione wcześniej, w taki sam sposób jak przy przetwarzaniu wsadowym. Ponownie mamy do dyspozycji interfejsy dla języka Scala, Java, Python oraz R, dzięki którym można tworzyć agregacje strumieniowe, okna zdarzeń w czasie, połączenia stream-to-batch i inne. Obliczenia są wykonywane na tym samym zoptymalizowanym silniku Spark SQL. Structured Streaming zapewnia szybkie, skalowalne, odporne na błędy, kompleksowe przetwarzanie strumieniowe dokładnie jeden raz (ang. exactly-once), bez konieczności zastanawiania się przez użytkownika nad sposobem przesyłania strumieniowego.

Wewnętrznie, domyślnie, zapytania Structured Streaming są przetwarzane przy użyciu silnika przetwarzania mikropartii, który przetwarza strumienie danych jako serię małych zadań wsadowych, osiągając w ten sposób opóźnienia end-to-end tak niskie, jak 100 milisekund i gwarancje tolerancji błędów exactly-once. Jednak od wersji Spark 2.3 wprowadzono nowy tryb przetwarzania o niskim opóźnieniu o nazwie Continuous Processing, który może osiągnąć opóźnienia end-to-end tak niskie jak 1 milisekunda z gwarancjami at-least-once. Bez zmiany operacji Dataset/DataFrame w zapytaniach, można wybrać tryb w oparciu o wymagania aplikacji.

# 1. Przykład użycia Structured Streaming.

## 1.1 Konfiguracja wstępna.

W przykładach wykorzystywane jest narzędzie `netcat`, które nie jest aktualnie zainstalowane w naszym obrazie dockerowym Ubuntu.

**Krok 1.**

Dodajemy do pliku `Dockerfile` następujące linie przed linią `USER spark`:
```bash

...

RUN apt-get update; \
	apt-get install -y netcat;

# poniższa linia już w pliku się znajduje
USER spark
```

**Krok 2**

Budujemy obraz ponownie co spowoduje dodanie kolejnej warstwy do istniejącego obrazu, ale zapiszemy go pod nową nazwą, tak na wszelki wypadek, gdyby coś poszło nie tak.

Uruchamiamy terminal w systemie hosta i ustawiamy ścieżkę na folder, który zawiera zmodyfikowany w kroku 1 plik Dockerfile. Następnie uruchamiamy polecenie:
```bash
docker build -t nowa_nazwa_obrazu:tag .
```

**Krok 3**

Teraz należy zmodyfikować plik `docker-compose.yml` i wskazać nowę nazwę obrazu, a którego obraz ma być tworzony, np.

```yml
# stara wersja
services:
  spark-master-3.5.3:
    image: spark-3.5.3:v3
    ...

# nowa wersja
services:
  spark-master-3.5.3:
    image: spark-3.5.3:v4 (tu nazwa nowego obrazu)
    ...
```

**Krok 4**

Uruchamiamy kontener poleceniem `docker-compose up` i sprawdzamy w konsoli kontenera czy polecenie `nc` jest już dostępne.

## 1.2 Kod i sposób uruchomienia prostego przykładu przetwarzania strumieniowego

**UWAGA !**

Jeżeli nie chcesz blokować notebooka (ten kod spowoduje uruchomienie przetwarzania w tle, ale będzie ono działało póki nie zostanie przerwane) powinno się uruchomić przykład w oddzielnym oknie terminala wewnątrz kontenera zgodnie z przykładem podanym po ostatnim fragmencie skryptu.

Poniżej zaprezentowany został przykład działania Structured Streaming zaczerpnięty z oficjalnej dokumentacji znajdującej się w poradniku: https://spark.apache.org/docs/3.5.1/structured-streaming-programming-guide.html

In [ ]:
# inicjalizacja sesji Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredStreamingExample") \
    .getOrCreate()

# domyślnie log level to INFO, które dość mocno spamuje okno konsoli
# możemy więc zmienić na nasze potrzeby do poziomu WARN
spark.sparkContext.setLogLevel('WARN')

In [ ]:
spark

In [ ]:
# tworzymy DataFrame reprezentujący strumień linii z połączenia do localhost:9999
# w dokumentacji znajdziemy też informację, że format socket jest wykorzystywany tylko
# do testowania

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# dzielenie linii na słowa
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# uruchomienie mechanizmu dzielenia tekstu na słowa
wordCounts = words.groupBy("word").count()

Teraz pozostaje włączenie mechanizmu pobierania danych i zliczania słów. Włączamy wyświetlanie całkowitej liczby zliczeń (ustawiane przez `outputMode("complete")`) na konsolę za każdym razem jak ta wartość zostanie zaktualizowana i uruchamiamy obliczenia strumieniowe poprzez metodę `start()`

Metoda `awaitTermination()` powoduje wykonanie tej akcji tylko wtedy, gdy obliczenia są już zakończone, a nie w ich trakcie.

In [ ]:
# uruchomienie zapytania, które będzie wyświetlało liczbę słów na konsolę
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

**UWAGA !**

Aby przetwarzanie strumieniowe zadziałało poprawnie musimy najpierw uruchomić poniższe polecenie, które uruchomi narzędzie `netcat`, które będzie nasłuchiwać na porcie 9999 póki nie zakończymy tego procesu. Wykonujemy to w oddzielnym oknie terminala naszego kontenera.

```bash
nc -lk 9999
```

In [ ]:
# możemy też zebrać całość w jeden skrypt i uruchomić z poziomu konsoli, co nie będzie blokowało możliwości
# wykonywania innych komórek notatnika, ale rekomendowane jest wywołanie komendy w terminalu
# wewnątrz kontenera ze względu na znaczną objętość outputu tej komórki oraz naturę tej "aplikacji"
# UPDATE: możemy jednak zmienić poziom logów na WARN co poprawia trochę sytuację w notebooku
# patrz plik źródłowy: structured_streaming_example.py
! ../../bin/spark-submit ./structured_streaming_example.py localhost 9999

# prawdopodobna postać polecenia w terminalu kontenera
/opt/spark/bin/spark-submit ./lab_12/structured_streaming_example.py localhost 9999

## 1.3 Jak to działa?

Poniżej przedstawiona zostanie koncepcja działania przetwarzania strumieniowego z użyciem Spark Structored Streaming oparta na oficjalnej dokumentacji.

Utworzony w powyższym zadaniu obiekt DataFrame oraz kolejne jego części, które są przetwarzane strumieniowo jako mikro paczki (ang. micro-batch) i stopniowo dołączane do ramki w kolejności napływania danych w strumieniu. Wizualizacja została przedstawiona na poniższej grafice.

![Streamin processing](./images/structured-streaming-stream-as-a-table.png)

W naszym przypadku na wyjściu otrzymujemy stan 'completed', ale możliwe jest określenie innych strategii zwracania danych z ramki. Poniżej opis za dokumentacją:

* `Complete Mode` - cała zaktualizowana tabela wyników zostanie zapisana w pamięci zewnętrznej. Decyzja o sposobie obsługi zapisu całej tabeli należy do konektora pamięci masowej.

* `Append Mode` - tylko nowe wiersze dodane do tabeli wyników od ostatniego wyzwalacza zostaną zapisane w zewnętrznej pamięci masowej. Ma to zastosowanie tylko w przypadku zapytań, w których nie oczekuje się zmiany istniejących wierszy w tabeli wyników.

* `Update Mode` - tylko wiersze, które zostały zaktualizowane w tabeli wyników od ostatniego wyzwalacza zostaną zapisane w pamięci zewnętrznej (dostępne od wersji Spark 2.1.1). Należy pamiętać, że ten tryb różni się od trybu `Complete`, ponieważ wyświetla tylko wiersze, które uległy zmianie od ostatniego wyzwalacza. Jeśli zapytanie nie zawiera agregacji, będzie to równoważne z trybem `Append`.

Nasz przykład wykorzystuje strumień oparty o gniazdo (ang. socket), który rekomendowany jest tylko do testów. Poniżej wymienione zostały inne możliwe wbudowane źródła danych strumieniowych dla obiektów Dataset/DataFrame i Structured Streaming:

* `źródło plikowe (file source)` - odczytuje pliki ze wskazanego katalogu jako dane strumieniowe. Pliki są przetwarzane w kolejności zgodnej z ich datą modyfikacji. Można tę kolejność odwrócić w zależności od potrzeb (parametr latestFirst). Wspierane formaty plików to: pliki tekstowe, CSV, JSON, ORC, Parquet.

* `źródło Apache Kafka` - Odczytywanie danych ze strumienia Apache Kafka. Więcej informacji w poradniku integracji Spark i Kafka dostępnym [tu](https://spark.apache.org/docs/3.5.1/structured-streaming-kafka-integration.html).

* `źródło gniazda (socket source)`, tylko do testowania - odczytuje tekst w formacie UTF8 ze źródła w postaci gniazda, a serwer nasłuchujący znajduje się na driverze klastra Spark. Nie zapewnia mechanizmu gwarancji and-to-end fault-tolerance.

* `żródło rate`, tylko testowanie - generuje dane z określoną liczbą wierszy na sekundę, każdy wiersz wyjściowy zawiera znacznik czasu i wartość. Znacznik czasu jest typem Timestamp zawierającym czas wysłania wiadomości, a value jest typem Long zawierającym liczbę wiadomości, zaczynając od 0 jako pierwszy wiersz. To źródło jest przeznaczone do testowania i analizy wydajności.

* `źródło Rate Per Micro-Batch`, tylko testowanie - generuje dane z określoną liczbą wierszy na sekundę, każdy wiersz wyjściowy zawiera znacznik czasu i wartość. Znacznik czasu jest typem Timestamp zawierającym czas wysłania wiadomości, a value jest typem Long zawierającym liczbę wiadomości, zaczynając od 0 jako pierwszy wiersz. W przeciwieństwie do źródła danych rate, to źródło danych zapewnia spójny zestaw wierszy wejściowych na mikropartię niezależnie od wykonania zapytania (konfiguracja wyzwalacza, opóźnienie zapytania itp.), powiedzmy, partia 0 wygeneruje 0\~999, a partia 1 wygeneruje 1000\~1999 itd. To samo dotyczy wygenerowanego czasu. To źródło jest przeznaczone do testowania i analizy wydajności.

## 1.4 Inne przykłady ramek strumieniowych.

Ponownie przykład ze strumieniem z gniazda. Jeżeli w tle nadal działa narzędzie netcat i do strumienia zostały wysłane jakieś dane, możemy je odczytać lub obserwować tutaj aktualizację ramki jeżeli do strumienia trafią nowe dane.

In [ ]:
# do uruchomienia tych przykładów potrzebne jest wcześniejsze stworzenie sesji Spark
socketDF = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [ ]:
socketDF.isStreaming

In [ ]:
socketDF.printSchema()

In [ ]:
# Write the streaming DataFrame to a table
socketDF.writeStream \
    .option("checkpointLocation", "./data/socstream/") \
    .toTable("socketTable")

# Check the table result
spark.read.table("socketTable").show()

In [ ]:
# po wysłaniu kilku wartości w terminalu z uruchomionym netcatem możemy ponownie odczytać
# wartości z tabeli utworzonej ze źródła strumieniowego
spark.read.table("socketTable").show()

In [ ]:
# po tej operacji są tworzone również pliki tymczasowe ukryte i zatrzymanie oraz wznowienie tego
# strumienia nie jest możliwe, jeżeli nie posprzątamy w tym folderze
# poniższe polecenie pomoże nam to zrobić
!rm -rf ./data/socstream/*
!rm -rf ./spark-warehouse/*

In [ ]:
# jeżeli uruchomiliśmy wcześniej kontekst Sparka, możemy go zakończyć
spark.stop()